In [ ]:
import sys
sys.path.append('/home/samer/projects/fuzzy_sql/src')

from fuzzy_sql.fuzzy_sql import *
import json
import os
from pathlib import Path


In [ ]:
# set directories
root_dir = Path('/home/samer/projects/fuzzy_sql')
metadata_dir = os.path.join(root_dir, 'data/sdgd/processed/metadata')
real_dir = os.path.join(root_dir, 'data/sdgd/processed/real')
syn_dir = os.path.join(root_dir, 'data/sdgd/processed/synthetic')
db_path = os.path.join(root_dir, 'db/sdgd.db')

In [ ]:
#Import testing set
with open ('tabular_test_set.json', 'r') as f:
    test_dict=json.load(f)

In [ ]:
# connect to db
conn = sqlite3.connect(db_path)

In [ ]:
# #Slice dictonary for development
# selected_datasets=['C1']
# test_dict=dict([(key, value) for key,value in test_dict.items() if key in selected_datasets])

In [ ]:
            # "type": "twin_agg",
            # "agg_fntn": agg_fntn_terms,
            # "grpby_vars": real_groupby_lst,
            # "from_tbl_name_real": real_from_tbl,
            # "join_tbl_name_lst_real": real_join_tbl_lst,
            # "sql_real": real_expr,
            # "n_cols_real": query_real.shape[1],
            # "n_rows_real": query_real.shape[0],
            # "from_tbl_name_syn": syn_from_tbl,
            # "join_tbl_name_lst_syn": syn_join_tbl_lst,
            # "sql_syn": syn_expr,
            # "n_cols_syn": query_syn.shape[1],
            # "n_rows_syn": query_syn.shape[0],
            
            
            
            
            
    # cur = db_conn.cursor()
    # cur.execute("SELECT count(name) FROM sqlite_master WHERE type='table' AND name=(?) ",
    #             (table_name,))  # sqlite_master holds  the schema of the db including table names
    # # If table does not exist (ie returned count is zero), then import the table into db from pandas
    # if cur.fetchone()[0] == 0: #if table does not exist
    #     df.to_sql(table_name, db_conn, index=False)
    #     print(f'Table {table_name} is created in the database')

In [ ]:
#generate test_twin lists
no_queries_per_twin=10
dist_catcher={
    'real_table_name': [],
    'syn_table_name':[],
    'no_data_vars':[],
    'no_query_agg_vars':[],
    'no_real_query_records':[],
    'no_syn_query_records':[],
    'query_hlngr_score':[],
    'query_ecldn_score':[]
}

for real_table_name in test_dict.keys():
    syn_list=test_dict[real_table_name]
    real_list=[real_table_name for i in range(len(syn_list))]
    with open(os.path.join(metadata_dir, real_table_name+'.json'), 'r') as f:
        meta=json.load(f)
    meta_list = [meta for i in range(len(syn_list)) ]
    
    cur = conn.cursor() #check no of variables in the dataset table
    data=cur.execute(f"SELECT * FROM {real_table_name}") 
    data=data.fetchall()
    

    #generate random queries
    for real, meta, syn in zip(real_list,meta_list,syn_list):
        print("\n******************************")
        print(f"\n {real} \n {syn}")
        print("\n******************************")
        random_queries=gen_queries(no_queries_per_twin,conn, [real], [meta],  [syn] )
        for i, query in enumerate(random_queries):
            dist_catcher['real_table_name'].append(real)
            dist_catcher['syn_table_name'].append(syn)
            dist_catcher['no_data_vars'].append(len(data[0]))
            dist_catcher['no_query_agg_vars'].append(len(random_queries[i]['query_desc']['grpby_vars']))
            dist_catcher['no_real_query_records'].append(random_queries[i]['query_desc']['n_rows_real'])
            dist_catcher['no_syn_query_records'].append(random_queries[i]['query_desc']['n_rows_syn'])
            dist_catcher['query_hlngr_score'].append(random_queries[i]['query_hlngr_score'])
            dist_catcher['query_ecldn_score'].append(random_queries[i]['query_ecldn_score'])
            

In [ ]:
random_queries